In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math

# Data loading

In [2]:
train_set = pd.read_csv('mnist_train.csv',header=None)
train_set = np.array(train_set)
train_set_data = train_set[:,1:]
train_set_label = train_set[:,0]
test_set = pd.read_csv('mnist_test.csv',header=None)
test_set = np.array(test_set)
test_set_data = test_set[:,1:]
test_set_label = test_set[:,0]

In [3]:
train_set_data_normalized = train_set_data/255.0
test_set_data_normalized = test_set_data/255.0 

# RBM class 

In [4]:
class RBM:
    def __init__(self,input_layer_size,hidden_layer_size):
        self.input_layer_array = np.zeros(input_layer_size)
        self.hidden_layer_array = np.zeros(hidden_layer_size)
        self.weight_matrix_array = np.random.rand(input_layer_size, hidden_layer_size)/np.sqrt(input_layer_size)
        self.input_bias_array = np.random.rand(input_layer_size, 1)/np.sqrt(input_layer_size)
        self.hidden_bias_array = np.random.rand(hidden_layer_size, 1)/np.sqrt(input_layer_size)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward_pass(self, v_layer_array):
        v_layer_array_temp = v_layer_array.copy()
        h_layer_array_temp = np.matmul(np.transpose(self.weight_matrix_array), v_layer_array_temp) \
                            + self.hidden_bias_array
        h_layer_array = self.sigmoid(h_layer_array_temp)
        del h_layer_array_temp
        del v_layer_array_temp
        return h_layer_array
    
    def sample(self, p_a_given_b_dist):
        dim = p_a_given_b_dist.shape[0]
        true_idx = np.random.uniform(0, 1, dim).reshape(dim, 1) <= p_a_given_b_dist
        sampled = np.zeros((dim, 1))
        sampled[true_idx] = 1  
        return sampled
            

# Classifier

In [5]:
class softmax_classifier():
    
    def __init__(self,architecture,pretrained=False,model_path=None):
        
        if pretrained is False:
            self.bias_matrix = [np.random.randn(number_of_neuron)/math.sqrt(architecture[0]) for number_of_neuron in architecture[1:]]
            self.weights_matrix = [(np.random.randn(neuron_in_next_layers, neuron_in_previous_layers)/math.sqrt(architecture[0]))\
                                   for neuron_in_previous_layers, neuron_in_next_layers in zip(architecture[:-1], architecture[1:])]
        else:
            with open(model_path+'model_weight.pkl','rb') as f:
                self.weights_matrix = pickle.load(f)
            with open(model_path+'model_bais.pkl','rb') as b:
                self.bias_matrix = pickle.load(b)
            
        self.num_layers = len(architecture)
        self.architecture = architecture
    
    def fc_layer(self,input_):
        temp = input_.copy()
        count = 0 
        for bais,weights in zip(self.bias_matrix,self.weights_matrix):
            if count<len(bais)-1:
                temp = self.relu(np.matmul(weights,temp)+bais)
            else:
                temp = self.softmax(np.matmul(weights,temp)+bais)
            count += 1
        return temp 
    
    def relu(self,vector):
        temp = vector.copy()
        return np.where(temp>=0,temp,0.0)
    
    def softmax(self,vector):
        temp = vector.copy()
        temp = np.exp(temp)
        return temp/np.sum(temp)
    
    def cross_entropy(self,vector,label):
        temp1 = vector.copy()
        temp2 = np.zeros(temp1.shape[0])
        temp2[int(label)] = 1
        temp1 = np.clip(temp1,1e-2,1-(1e-2))
        return -1*np.sum(np.log(temp1)*temp2)/temp1.shape[0]
    
    def backpropagation(self, image, gt_truth):
        image_delta_w = [np.zeros((neuron_in_next_layers, neuron_in_previous_layers))\
                               for neuron_in_previous_layers, neuron_in_next_layers in zip(self.architecture[:-1], self.architecture[1:])]
        image_delta_b = [np.zeros(number_of_neuron) for number_of_neuron in self.architecture[1:]]
        
        temp = image.copy()
        z_layers=[]
        activation_layers=[]
        activation_layers.append(temp)
        count = 0 
        for bais,weights in zip(self.bias_matrix,self.weights_matrix):
            temp = np.matmul(weights,temp)+bais
            z_layers.append(temp)
            if count<len(bais)-1:
                temp = self.relu(temp)
            else:
                temp = self.softmax(temp)
                
            activation_layers.append(temp)
            count += 1
        
        one_hot_vector_label = np.zeros(temp.shape[0])
        one_hot_vector_label[int(gt_truth)] = 1
        del_cost_activation = temp - one_hot_vector_label
        image_delta_w[-1] = np.matmul(del_cost_activation.reshape((-1,1)), activation_layers[-2].reshape((1,-1)))
        image_delta_b[-1] = del_cost_activation.copy()
    
        
        for i in range(2,len(image_delta_w)+1):
            del_cost_activation_temp = del_cost_activation.copy()
            del_cost_activation = np.squeeze(np.matmul(del_cost_activation_temp.reshape((1,-1)),self.weights_matrix[-1*i+1]))
            del_cost_activation = np.where(z_layers[-1*i]>=0, del_cost_activation, 0)
            image_delta_w[-1*i] = np.matmul(del_cost_activation.reshape((-1,1)), activation_layers[-1*i-1].reshape((1,-1)))
            image_delta_b[-1*i] = del_cost_activation.copy()
        
        return image_delta_w, image_delta_b
    
    def train(self,train_data=[],number_of_epochs=5,learning_rate=0.01,batch_size=500,test_data=[],lamda=0.1,save_path = None):
        data = train_data.copy()
        previous_loss = float('Inf')
        train_continue = True
        early_stopping = np.zeros(3)
        count = 0
        iter_ = 0
#         for epoch in range(number_of_epochs):
        while(train_continue==True):
            correct_prediction = 0
            training_loss = 0
            np.random.shuffle(data)
            
            for image in data:
                output_softmax = self.fc_layer(image[1:])
                training_loss += self.cross_entropy(output_softmax,image[0])
                output = np.argmax(output_softmax)
                if output == image[0]:
                    correct_prediction += 1
                    
                Training_Accuracy = (correct_prediction/data.shape[0])*100
            
            if test_data!=[]:
                correct_prediction = 0
                testing_loss = 0
                for image in test_data:
                    output_softmax = self.fc_layer(image[1:])
                    testing_loss += self.cross_entropy(output_softmax,image[0])
                    output = np.argmax(output_softmax)
                    if output == image[0]:
                        correct_prediction += 1
                Testing_Accuracy = (correct_prediction/test_data.shape[0])*100
                
                if testing_loss<previous_loss:
                    previous_loss = testing_loss
                    with open(save_path+'model_weight.pkl','wb') as f:
                        pickle.dump(self.weights_matrix, f)
                    with open(save_path+'model_bais.pkl','wb') as b:
                        pickle.dump(self.bias_matrix, b)
                
                if (testing_loss-previous_loss)>=0.1:
                    early_stopping[count%3] = 1
                else:
                    early_stopping[count%3] = 0
                
                if np.sum(early_stopping) == 3:
                    train_continue = False
                    
                count+=1
                
#                 print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3))
                print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3), ', Train Accuracy:', round(Training_Accuracy,3),'%',', Test Accuracy:', round(Testing_Accuracy,3),'%')
            
            else:
                print('epoch:',epoch ,', Train Loss:', round(training_loss,3))
                #print('epoch:',epoch ,', Train Loss:', round(training_loss,3),', Train Accuracy:', round(Training_Accuracy,3),'%')
            
            iter_ += 1
            if iter_ > number_of_epochs:
                train_continue = False
            
            
            for batch_number in range(data.shape[0]//batch_size):
                
                batch_delta_w = [np.zeros((neuron_in_next_layers, neuron_in_previous_layers))\
                                 for neuron_in_previous_layers, neuron_in_next_layers in zip(self.architecture[:-1], self.architecture[1:])]
                batch_delta_b = [np.zeros(number_of_neuron) for number_of_neuron in self.architecture[1:]]
                
                for image in data[batch_size*batch_number:batch_size*(batch_number+1)]:
                    image_delta_w, image_delta_b = self.backpropagation(image[1:], image[0])
                
                    batch_delta_w = [batch_weight_delta+image_weight_delta for batch_weight_delta, image_weight_delta in zip(batch_delta_w, image_delta_w)]
                    batch_delta_b = [batch_bais_delta+image_bais_delta for batch_bais_delta, image_bais_delta in zip(batch_delta_b, image_delta_b)]
                    
                
                self.weights_matrix = [weight_layer-(learning_rate/batch_size)*weight_layer_delta + lamda*weight_layer
                            for weight_layer, weight_layer_delta in zip(self.weights_matrix, batch_delta_w)]
                self.bias_matrix = [bais_layer-(learning_rate/batch_size)*bais_layer_delta
                           for bais_layer, bais_layer_delta in zip(self.bias_matrix, batch_delta_b)]
            
                del batch_delta_w, batch_delta_b, image_delta_w, image_delta_b
    
    def evaluate(self,test_data):
        testing_loss = 0
        correct_prediction = 0
        for image in test_data:
            output_softmax = self.fc_layer(image[1:])
            testing_loss += self.cross_entropy(output_softmax,image[0])
            output = np.argmax(output_softmax)
            if output == image[0]:
                correct_prediction += 1
        print('Test Loss:', round(testing_loss,3),', Test Accuracy:', round((correct_prediction/test_data.shape[0])*100,3),'%')
            
       

In [74]:
np.random.seed(1)
flattened_image_classifier = softmax_classifier([784,10])
flattened_image_classifier.train(train_data=np.concatenate((train_set_label.reshape(-1,1),train_set_data_normalized),axis=1), number_of_epochs=50, learning_rate=0.01, batch_size=10, test_data=np.concatenate((test_set_label.reshape(-1,1),test_set_data_normalized),axis=1),lamda=0.0,save_path='model/flattened')

/home/saarthak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


epoch: 1 , Train Loss: 16921.668 , Test Loss: 2763.228 , Train Accuracy: 8.253 % , Test Accuracy: 8.36 %
epoch: 2 , Train Loss: 2924.281 , Test Loss: 467.849 , Train Accuracy: 90.242 % , Test Accuracy: 90.37 %
epoch: 3 , Train Loss: 2961.187 , Test Loss: 475.901 , Train Accuracy: 90.767 % , Test Accuracy: 90.88 %
epoch: 4 , Train Loss: 2859.145 , Test Loss: 466.048 , Train Accuracy: 90.845 % , Test Accuracy: 90.65 %
epoch: 5 , Train Loss: 2591.852 , Test Loss: 422.359 , Train Accuracy: 91.215 % , Test Accuracy: 91.29 %
epoch: 6 , Train Loss: 2711.241 , Test Loss: 444.581 , Train Accuracy: 91.37 % , Test Accuracy: 91.31 %
epoch: 7 , Train Loss: 2639.521 , Test Loss: 434.871 , Train Accuracy: 91.407 % , Test Accuracy: 91.45 %
epoch: 8 , Train Loss: 2604.231 , Test Loss: 427.761 , Train Accuracy: 91.597 % , Test Accuracy: 91.54 %


In [8]:
flattened_test_network = softmax_classifier([784,10],pretrained=True,model_path='model/flattened')
flattened_test_network.evaluate(test_data=np.concatenate((test_set_label.reshape(-1,1),test_set_data_normalized),axis=1))

Test Loss: 422.359 , Test Accuracy: 91.29 %


In [6]:
rbm_trained = RBM(28 * 28, 100)

weight_file = open(b"model/weight_matrix.npy","rb")
input_bias_file = open(b"model/input_bias_array.npy","rb")
hidden_bias_file = open(b"model/hidden_bias_array.npy","rb")

rbm_trained.weight_matrix_array = pickle.load(weight_file)
rbm_trained.input_bias_array = pickle.load(input_bias_file)
rbm_trained.hidden_bias_array = pickle.load(hidden_bias_file)

In [7]:
rbm_features_dataset_train = np.zeros((train_set_data_normalized.shape[0],100))
rbm_features_dataset_test = np.zeros((test_set_data_normalized.shape[0],100))

for i in range(train_set_data_normalized.shape[0]):
    
    v_0 = train_set_data_normalized[i].copy()
    v_0 = v_0.reshape(-1,1)
    prob_h_given_v = rbm_trained.forward_pass(v_0)
#     h_0 = rbm_trained.sample(prob_h_given_v)
    h_0 = prob_h_given_v
    rbm_features_dataset_train[i] = h_0.reshape(-1)
    
for i in range(test_set_data_normalized.shape[0]):
    
    v_0 = test_set_data_normalized[i].copy()
    v_0 = v_0.reshape(-1,1)
    prob_h_given_v = rbm_trained.forward_pass(v_0)
#     h_0 = rbm_trained.sample(prob_h_given_v)
    h_0 = prob_h_given_v
    rbm_features_dataset_test[i] = h_0.reshape(-1)

In [83]:
np.random.seed(1)
rbm_features_classifier = softmax_classifier([100,10])
rbm_features_classifier.train(train_data=np.concatenate((train_set_label.reshape(-1,1),rbm_features_dataset_train),axis=1), number_of_epochs=50, learning_rate=0.01, batch_size=10, test_data=np.concatenate((test_set_label.reshape(-1,1),rbm_features_dataset_test),axis=1),lamda=0.0,save_path='model/rbm_features')

/home/saarthak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


epoch: 1 , Train Loss: 21852.021 , Test Loss: 3662.848 , Train Accuracy: 8.835 % , Test Accuracy: 8.25 %
epoch: 2 , Train Loss: 2976.096 , Test Loss: 476.294 , Train Accuracy: 89.448 % , Test Accuracy: 90.19 %
epoch: 3 , Train Loss: 2713.839 , Test Loss: 433.451 , Train Accuracy: 89.972 % , Test Accuracy: 90.69 %
epoch: 4 , Train Loss: 2576.395 , Test Loss: 411.925 , Train Accuracy: 90.24 % , Test Accuracy: 90.86 %
epoch: 5 , Train Loss: 2533.528 , Test Loss: 405.285 , Train Accuracy: 90.422 % , Test Accuracy: 91.05 %
epoch: 6 , Train Loss: 2489.788 , Test Loss: 398.734 , Train Accuracy: 90.523 % , Test Accuracy: 91.12 %
epoch: 7 , Train Loss: 2433.606 , Test Loss: 389.402 , Train Accuracy: 90.698 % , Test Accuracy: 91.18 %
epoch: 8 , Train Loss: 2407.984 , Test Loss: 386.156 , Train Accuracy: 90.702 % , Test Accuracy: 91.11 %
epoch: 9 , Train Loss: 2386.268 , Test Loss: 382.748 , Train Accuracy: 90.695 % , Test Accuracy: 91.23 %
epoch: 10 , Train Loss: 2401.354 , Test Loss: 385.591 , 

In [9]:
rbm_test_network = softmax_classifier([100,10],pretrained=True,model_path='model/rbm_features')
rbm_test_network.evaluate(test_data=np.concatenate((test_set_label.reshape(-1,1),rbm_features_dataset_test),axis=1))

Test Loss: 369.092 , Test Accuracy: 91.33 %
